In [16]:
import numpy as np
import open3d as o3d
from pykinect2 import PyKinectV2
from pykinect2 import PyKinectRuntime

In [18]:
# Main Settings

# Angle between each scan in degrees
ANGLE_INCR: int = 45

MIN_DEPTH: int = 750
MAX_DEPTH: int = 1250

MIN_X: int = 120
MAX_X: int = 500

MIN_Y: int = 0
MAX_Y: int = 400

# The target object type.
TARGET: str = 'sphere'

In [19]:
# Initialize Kinect

kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth)

In [20]:
# Initialize Pointclouds Container

pcds = []

In [ ]:
#viz = o3d.visualization.Visualizer()
#viz.create_window(window_name='KinectScan', width=720, height=480, visible=True)

In [21]:
for angle in range(0, 360, ANGLE_INCR):
        

    depth_frame = np.reshape(kinect.get_last_depth_frame(), (424, 512)).astype(np.uint16)

    # Filter Image
    depth_frame = np.where((depth_frame >= MIN_DEPTH) & (depth_frame <= MAX_DEPTH), depth_frame, 0)
    depth_frame[:MIN_Y, :] = 0
    depth_frame[MAX_Y:, :] = 0
    depth_frame[:, :MIN_X] = 0
    depth_frame[:, MAX_X:] = 0

    depth_image = o3d.geometry.Image(depth_frame)

    # Create Pointcloud
    pcd = o3d.geometry.PointCloud.create_from_depth_image(
        depth_image,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.Kinect2DepthCameraDefault
        )
    )

        # Rotate Pointcloud

    print(f'Rotating Pointcloud by {angle} degrees...')
    rotation = o3d.geometry.get_rotation_matrix_from_xyz((0, angle, 0))
    pcd.rotate(rotation)

    # Add Pointcloud to Container

    print("Writing to File")
    o3d.io.write_point_cloud(f'../data/Kinect/{TARGET}/raw/{angle}.ply', pcd)

    pcds.append(pcd)
    #viz.add_geometry(pcd)

    # Update Visualization
    #viz.update_renderer()
    #viz.poll_events()

    # wait input
    input("Press Enter to continue...")


kinect.close()
#viz.destroy_window()

Rotating Pointcloud by 0 degrees...
Writing to File
Rotating Pointcloud by 45 degrees...
Writing to File
Rotating Pointcloud by 90 degrees...
Writing to File
Rotating Pointcloud by 135 degrees...
Writing to File
Rotating Pointcloud by 180 degrees...
Writing to File
Rotating Pointcloud by 225 degrees...
Writing to File
Rotating Pointcloud by 270 degrees...
Writing to File
Rotating Pointcloud by 315 degrees...
Writing to File


In [ ]:
del kinect

In [22]:
# Register Collected Pointclouds

assert len(pcds) > 0, "-- No Pointclouds Collected --"

final_pcd = pcds[0]
for pcd in pcds[1:]:
    icpRegResults = o3d.pipelines.registration.registration_icp(
        pcd, 
        final_pcd, 
        0.1, 
        np.identity(4), 
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
    )

    pcd.transform(icpRegResults.transformation)
    final_pcd += pcd

In [23]:
# Visualize Result

o3d.visualization.draw_geometries([final_pcd])

In [ ]:
# Save Result

o3d.io.write_point_cloud(f'../data/Kinect/{TARGET}/result.ply', final_pcd)